                                            
# Lecture 14                    
                                            
## Binary/Probability models                 
  - Saturated Probability Models          
    - calculate weights and               
        estimating models                 
    - plot a simple binary model          
    - predicted and true outcome          
  - Functional form decision              
  - Multiple Regression w binary outcome  
    - characteristics of predicted groups 
  - Logit and Probit models               
    - simple estimates                    
    - average marginal effects            
  - Model comparison of non-linear models 
    - Goodness-of-fit statistics:         
    R2, Pseudo-R2, Brier score, Log-loss  
    - Visual inspection:                  
    distribution of prediction by outcome 
    - summary stats for predictions       
        by outcome                        
    - Bias and calibration curve          
                                            
                                            
#### Case Study
  - Does Smoking Pose a Health Risk           
                                            
#### Dataset used                             
  - share-health                 
---

Import packages

In [1]:
import warnings

import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from mizani.formatters import percent_format
from plotnine import *
from skimpy import skim
from stargazer.stargazer import Stargazer

%matplotlib inline
warnings.filterwarnings("ignore")

Import data

In [2]:
share = pd.read_csv("https://osf.io/3ze58/download")
share.head()

,ID,wave,country,country_mod,int_year,int_month,female,age,sphus,br015,...,bmi,mar_stat,healthy,baseline,endline,stayshealthy,married,exerc,eduyears,country_str
0,1,4,11,40,2011,9,0,59.799999,2,3,...,23.198315,1,1,1,0,1,1,0.0,13.0,Austria
1,2,4,11,40,2011,8,1,58.200001,1,2,...,24.801588,1,1,1,0,1,1,0.0,14.0,Austria
2,3,4,11,40,2011,9,1,57.000000,2,4,...,22.038567,3,1,1,0,1,0,0.0,23.0,Austria
3,4,4,11,40,2011,12,0,59.299999,2,1,...,25.826447,1,1,1,0,0,1,1.0,12.0,Austria
4,5,4,11,40,2011,9,0,53.299999,2,1,...,26.827421,1,1,1,0,0,1,1.0,3.0,Austria


Check all the data - quick description 

(All except country_str, but it is implied in country_mod)

In [4]:
skim(share)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 3272   │ │ int64       │ 18    │                                                          │
│ │ Number of columns │ 23     │ │ float64     │ 4     │                                                          │
│ └───────────────────┴────────┘ │ string      │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓  │
│ ┃ column_name         ┃ NA   ┃ NA %     ┃ mean    ┃ sd      ┃ p0      ┃ p25     ┃ p75    ┃ p100   ┃ hist     ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩  │
│ │ ID                  │    0 │        0 │    1600 │     940 │       1 │     820 │   2500 │   3300 │  ██████  │  │
│ │ wave                │    0 │        0 │       4 │       0 │       4 │       4 │      4 │      4 │      █   │  │
│ │ country             │    0 │        0 │      20 │     6.9 │      11 │      16 │     23 │     35 │  ▄█▃▃▂▃  │  │
│ │ country_mod         │    0 │        0 │     350 │     270 │      40 │      56 │    700 │    760 │  ▆█▁ ▁▆  │  │
│ │ int_year            │    0 │        0 │    2000 │    0.15 │    2000 │    2000 │   2000 │   2000 │      █   │  │
│ │ int_month           │    0 │        0 │     6.1 │     2.6 │       1 │       4 │      8 │     12 │  ▁▅█▆▃▂  │  │
│ │ female              │    0 │        0 │    0.57 │     0.5 │       0 │       0 │      1 │      1 │  ▆    █  │  │
│ │ age                 │    0 │        0 │      55 │     2.8 │      50 │      53 │     58 │     60 │  ▆▇▇██▇  │  │
│ │ sphus               │    0 │        0 │     1.7 │    0.46 │       1 │       1 │      2 │      2 │  ▃    █  │  │
│ │ br015               │    0 │        0 │       2 │     1.3 │     -12 │       1 │      3 │      4 │      █▇  │  │
│ │ smoking             │    0 │        0 │    0.22 │    0.42 │       0 │       0 │      0 │      1 │  █    ▂  │  │
│ │ ever_smoked         │    0 │        0 │    0.49 │     0.5 │       0 │       0 │      1 │      1 │  █    █  │  │
│ │ income10            │    0 │        0 │     6.8 │     2.8 │       1 │       5 │      9 │     10 │  ▂▁▃▂▆█  │  │
│ │ bmi                 │   75 │      2.3 │      26 │     3.9 │      16 │      23 │     28 │     46 │   ▂█▅▁   │  │
│ │ mar_stat            │    0 │        0 │       2 │     1.7 │     -12 │       1 │      3 │      6 │      █▃  │  │
│ │ healthy             │    0 │        0 │       1 │       0 │       1 │       1 │      1 │      1 │      █   │  │
│ │ baseline            │    0 │        0 │       1 │       0 │       1 │       1 │      1 │      1 │      █   │  │
│ │ endline             │    0 │        0 │       0 │       0 │       0 │       0 │      0 │      0 │      █   │  │
│ │ stayshealthy        │    0 │        0 │     0.6 │    0.49 │       0 │       0 │      1 │      1 │  ▅    █  │  │
│ │ married             │    0 │        0 │    0.74 │    0.44 │       0 │       0 │      1 │      1 │  ▃    █  │  │
│ │ exerc               │    1 │    0.031 │    0.52 │     0.5 │       0 │       0 │      1 │      1 │  ▇    █  │  │
│ │ eduyears            │   88 │      2.7 │      12 │   

Remove if any of important variable is missing 

In [5]:
share = (
    share.loc[share["bmi"].notnull()]
    .loc[share["eduyears"].notnull()]
    .loc[share["exerc"].notnull()]
)

Make descriptive statistics for thery based selected variables

In [6]:
share.filter(
    [
        "stayshealthy",
        "smoking",
        "ever_smoked",
        "female",
        "age",
        "income10",
        "eduyears",
        "bmi",
        "exerc",
    ]
).describe().T

,count,mean,std,min,25%,50%,75%,max
stayshealthy,3109.0,0.604696,0.488995,0.00000,0.000000,1.000000,1.000000,1.000000
smoking,3109.0,0.223866,0.416900,0.00000,0.000000,0.000000,0.000000,1.000000
ever_smoked,3109.0,0.492120,0.500018,0.00000,0.000000,0.000000,1.000000,1.000000
female,3109.0,0.560952,0.496351,0.00000,0.000000,1.000000,1.000000,1.000000
age,3109.0,55.187617,2.788072,50.00000,52.799999,55.299999,57.500000,60.000000
income10,3109.0,6.790286,2.745141,1.00000,5.000000,7.000000,9.000000,10.000000
eduyears,3109.0,12.330814,4.556630,0.00000,10.000000,13.000000,16.000000,25.000000
bmi,3109.0,25.531596,3.905711,16.32653,22.837370,25.099501,27.774271,45.632774
exerc,3109.0,0.523963,0.499506,0.00000,0.000000,1.000000,1.000000,1.000000


In [7]:
skim(
    share.filter(
        [
            "stayshealthy",
            "smoking",
            "ever_smoked",
            "female",
            "age",
            "income10",
            "eduyears",
            "bmi",
            "exerc",
        ]
    )
)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 3109   │ │ int64       │ 5     │                                                          │
│ │ Number of columns │ 9      │ │ float64     │ 4     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┓  │
│ ┃ column_name          ┃ NA    ┃ NA %     ┃ mean    ┃ sd      ┃ p0   ┃ p25    ┃ p75    ┃ p100    ┃ hist      ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━┩  │
│ │ stayshealthy         │     0 │        0 │     0.6 │    0.49 │    0 │      0 │      1 │       1 │  ▅    █   │  │
│ │ smoking              │     0 │        0 │    0.22 │    0.42 │    0 │      0 │      0 │       1 │  █    ▂   │  │
│ │ ever_smoked          │     0 │        0 │    0.49 │     0.5 │    0 │      0 │      1 │       1 │  █    █   │  │
│ │ female               │     0 │        0 │    0.56 │     0.5 │    0 │      0 │      1 │       1 │  ▆    █   │  │
│ │ age                  │     0 │        0 │      55 │     2.8 │   50 │     53 │     58 │      60 │  ▆▇▇██▇   │  │
│ │ income10             │     0 │        0 │     6.8 │     2.7 │    1 │      5 │      9 │      10 │  ▂▁▃▂▆█   │  │
│ │ eduyears             │     0 │        0 │      12 │     4.6 │    0 │     10 │     16 │      25 │  ▂▂██▄▁   │  │
│ │ bmi                  │     0 │        0 │      26 │     3.9 │   16 │     23 │     28 │      46 │   ▂█▅▁    │  │
│ │ exerc                │     0 │        0 │    0.52 │     0.5 │    0 │      0 │      1 │       1 │  ▇    █   │  │
│ └──────────────────────┴───────┴──────────┴─────────┴─────────┴──────┴────────┴────────┴─────────┴───────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

### Saturated Linear Probability Models

_y_ is stayshealthy

Linear probability models of good health at endline and smoking


 __1st model__: current smoker on RHS

In [ ]:
lpm1 = smf.ols("stayshealthy ~ smoking", data=share).fit(cov_type ="HC3")
print(lpm1.summary())

Get the predicted values

In [ ]:
share["pred1"] = lpm1.predict()

Compare smoking with predicted values and real outcomes

1) Predicted vs smoking


In [ ]:
pd.crosstab(index=share["pred1"], columns=share["smoking"])

In [ ]:
pd.crosstab(index = share['stayshealthy'],columns= share['smoking'])

Create weights for prettier plot

In [ ]:
share["weight"] = share.groupby(["smoking","stayshealthy"])["ID"].transform("count")
share["weight_2"] = share["weight"] / 1000

Show graph with actual and predicted probabilities: LPM


In [ ]:
(
    ggplot(share)
    + geom_point(aes(x="smoking", y="pred1"), size=1, color="blue")
    + geom_line(aes(x="smoking", y="pred1"), colour="blue", size=0.7)
    + geom_point(
        aes(x="smoking", y="stayshealthy", size="weight_2"),
        fill="red",
        color="red",
        alpha=0.8,
        na_rm=True,
    )
    + scale_size_continuous(guide=False)
    + labs(x="Current smoker", y="Staying healthy / Predicted probability of ")
    + coord_cartesian(xlim=(0, 1), ylim=(0, 1))
    + scale_y_continuous(limits=(0, 1), breaks=np.arange(0, 1, 0.1))
    + scale_x_continuous(limits=(0, 1), breaks=np.arange(0, 1, 1))
    + theme_bw()
)

__2nd model__: current smoker and ever smoked on RHS

In [ ]:
lpm2 = smf.ols("stayshealthy ~ smoking + ever_smoked", data=share).fit(cov_type ="HC3")
lpm2.summary()

Compare models

In [ ]:
table = Stargazer([lpm1,lpm2])
table

### Multiple variable regressions:   
    selecting multiple variables, when outcome is binary

 As usual: check some functional forms

 For pretty plots create weigths for education

In [ ]:
share["weight"] = (
    share.groupby(["eduyears", "stayshealthy"])["smoking"].transform(len) / 100
)

In [ ]:
g2 = (
    ggplot(data=share)
    + geom_point(
        aes(x="eduyears", y="stayshealthy", size="weight"),
        color="blue",
        alpha=0.8,
        na_rm=True,
    )
    + geom_smooth(
        aes(x="eduyears", y="stayshealthy"), method="loess", se=False, color="red"
    )
    + labs(x="Years of education", y="Probability of staying healthy ")
    + scale_x_continuous(
        expand=[0.01, 0.01], limits=[0, 25], breaks=np.arange(0, 25, 4)
    )
    + scale_y_continuous(
        expand=[0.01, 0.01], limits=[0, 1], breaks=np.arange(0, 1.1, 0.1)
    )
    + scale_size_continuous(guide=False)
    + theme_bw()
)
g2

In [ ]:
(
    ggplot(data=share)
    + geom_smooth(
        aes(x="income10", y="stayshealthy"), method="loess", se=False, color="red"
    )
    + labs(
        x="Income group within country (deciles)", y="Probability of staying healthy "
    )
    + scale_x_continuous(expand=[0.01, 0.01], limits=[1, 10], breaks=np.arange(0, 11, 2))
    + scale_y_continuous(expand=[0.01, 0.01], limits=[0, 1], breaks=np.arange(0, 1.1, 0.1))
    + theme_bw()
)

In [ ]:
(
    ggplot(data=share)
    + geom_smooth(
        aes(x="age", y="stayshealthy"), method="loess", se=False, color="red"
    )
    + scale_y_continuous(
        expand=[0.01, 0.01],
        limits=[0, 1],
        breaks=np.arange(0, 1.1, 0.2),
        labels=percent_format(),
    )
    + labs(x="Age at interview (years)", y="Probability of staying healthy")
    + theme_bw()
)

In [ ]:
(
    ggplot(data=share)
    + geom_smooth(aes(x="bmi", y="stayshealthy"), method="loess", se=False, color="red")
    + scale_y_continuous(
        expand=[0.01, 0.01], limits=[0, 1], breaks=np.arange(0, 1.1, 0.2)
    )
    + scale_x_continuous(limits=[10, 50], breaks=np.arange(10, 51, 10))
    + labs(x="Body mass index", y="Stays healthy")
    + theme_bw()
)

#### Task

Linear probability model with many covariates, use the following
     
     smoking, ever_smoked, female, age, eduyears, income10, bmi, exerc, country_str

Use the P.L.S transformations:
   - eduyears: with knots at 8 (elementary only) and 18 (Diploma)
   - bmi: with knot at 35

and include `country` as categories, statsmodel will create dummies aout of them

In [ ]:
share['country'] = share['country'].astype('category')

In [ ]:
import copy
def lspline(series, knots):
    def knot_ceil(vector, knot):
        vector_copy = copy.deepcopy(vector)
        vector_copy[vector_copy > knot] = knot
        return vector_copy

    if type(knots) != list:
        knots = [knots]
    design_matrix = None
    vector = series.values

    for i in range(len(knots)):
        # print(i)
        # print(vector)
        if i == 0:
            column = knot_ceil(vector, knots[i])
        else:
            column = knot_ceil(vector, knots[i] - knots[i - 1])
        # print(column)
        if i == 0:
            design_matrix = column
        else:
            design_matrix = np.column_stack((design_matrix, column))
        # print(design_matrix)
        vector = vector - column
    design_matrix = np.column_stack((design_matrix, vector))
    # print(design_matrix)
    return design_matrix

In [ ]:
lpm3 = smf.ols(
    "stayshealthy ~ smoking + ever_smoked + female + age + lspline(eduyears,[8,18]) + \
                    income10 + lspline(bmi,[35]) + exerc + country",
    share,
).fit(covtype="HC3")

In [ ]:
table = Stargazer([lpm1, lpm2, lpm3])
table.covariate_order(
    [
        "smoking",
        "ever_smoked",
        "female",
        "age",
        "lspline(eduyears, [8, 18])[0]",
        "lspline(eduyears, [8, 18])[1]",
        "lspline(eduyears, [8, 18])[2]",
        "income10",
        "lspline(bmi, [35])[0]",
        "lspline(bmi, [35])[1]",
        "exerc",
    ]
)
table

Check predicted probabilities: is there any interesting values?   

In [ ]:
share["pred_lpm"] = lpm3.predict()

Make a descriptive summary of the predictions with 3 digits

In [ ]:
share['pred_lpm'].describe().round(4)

Show the predicted probabilities' distribution

In [ ]:
(
    ggplot(share, aes(x="pred_lpm"))
    + geom_histogram(
        aes(y="stat(count)/sum(stat(count))"),
        binwidth=0.02,
        center=0.65,
        color="white",
        fill="blue",
        closed="right",
        na_rm=True
    )
    + labs(x="Predicted probability of staying healthy (LPM)", y="Percent")
    + scale_y_continuous(
        expand=[0.00, 0.0],
        limits=[0, 0.07],
        breaks=np.arange(0, 0.07, 0.01),
        labels=percent_format(),
    )
    + scale_x_continuous(
        expand=[0.001, 0.01], limits=[0, 1.1], breaks=np.arange(0, 1.1, 0.2)
    )
    + theme_bw()
)

 We are interested in the top 1% and bottom 1% characteristics!
    
   Is there any significant difference?


   Create bins which categorize the predicted values between 1-100


In [ ]:
cuts = 100
share["q100_pred_lpm"] = pd.qcut(share["pred_lpm"], q=cuts, labels=range(1, cuts + 1))

Top 1%

In [ ]:
share.loc[
    share["q100_pred_lpm"] == 100,
    ["smoking", "ever_smoked", "female", "age", "eduyears", "income10", "bmi", "exerc"],
].describe().round(1)


Bottom 1%

In [ ]:
share.loc[
    share["q100_pred_lpm"] == 1,
    ["smoking", "ever_smoked", "female", "age", "eduyears", "income10", "bmi", "exerc"],
].describe().round(1)

### LOGIT AND PROBIT MODELS

Lets compare:\
   lpm versus logit and probit
 
 with all right-hand-side variables

 If comparing different estimation methods for the same model setup:
   good practice to make a 'formula' variable!

 To have pretty outcomes, we need to create spline variables

In [ ]:
formula = "stayshealthy ~ smoking + ever_smoked + female + age + lspline(eduyears,[8,18]) + \
                 income10 + lspline(bmi,[35]) + exerc + country"

Repeat LPM with new formula

In [ ]:
lpm = smf.ols(formula, share).fit(covtype="HC3")

table = Stargazer([lpm])
table.covariate_order(
    [
        "smoking",
        "ever_smoked",
        "female",
        "age",
        "lspline(eduyears, [8, 18])[0]",
        "lspline(eduyears, [8, 18])[1]",
        "lspline(eduyears, [8, 18])[2]",
        "income10",
        "lspline(bmi, [35])[0]",
        "lspline(bmi, [35])[1]",
        "exerc",
    ]
)
table

Save predictions

In [ ]:
share["pred_lpm"] = lpm.predict()

Logit

In [ ]:
logit = smf.logit(formula, share).fit()


table = Stargazer([logit])
table.covariate_order(
    [
        "smoking",
        "ever_smoked",
        "female",
        "age",
        "lspline(eduyears, [8, 18])[0]",
        "lspline(eduyears, [8, 18])[1]",
        "lspline(eduyears, [8, 18])[2]",
        "income10",
        "lspline(bmi, [35])[0]",
        "lspline(bmi, [35])[1]",
        "exerc",
    ]
)
table

In [ ]:
share["pred_logit"] = logit.predict()

Calculate the Average Marginal Effects 

In [ ]:
logit.get_margeff().summary()

Probit

In [ ]:
probit = smf.probit(formula, share).fit()


table = Stargazer([probit])
table.covariate_order(
    [
        "smoking",
        "ever_smoked",
        "female",
        "age",
        "lspline(eduyears, [8, 18])[0]",
        "lspline(eduyears, [8, 18])[1]",
        "lspline(eduyears, [8, 18])[2]",
        "income10",
        "lspline(bmi, [35])[0]",
        "lspline(bmi, [35])[1]",
        "exerc",
    ]
)
table

In [ ]:
share["pred_probit"] = probit.predict()

probit marginal differences

In [ ]:
probit.get_margeff().summary()

Comparing predictions from the three models

In [ ]:
share.filter(["pred_lpm","pred_logit","pred_probit"]).describe().T

Creating a model summary output for base models

In [ ]:
table = Stargazer([lpm,logit,probit])
table.covariate_order(
    [
        "smoking",
        "ever_smoked",
        "female",
        "age",
        "lspline(eduyears, [8, 18])[0]",
        "lspline(eduyears, [8, 18])[1]",
        "lspline(eduyears, [8, 18])[2]",
        "income10",
        "lspline(bmi, [35])[0]",
        "lspline(bmi, [35])[1]",
        "exerc",
    ]
)
table

### Goodness of Fit (GoF) statistics 
       with binary models

 goodness of fit is the same for marginal effects 
   as the base models, as it only calculates some averaged effects.

Lets import some metrics from the sklearn package

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import log_loss

In [ ]:
pd.DataFrame(
    {
        "R-squared": [
            lpm.rsquared,
            r2_score(share["stayshealthy"], share["pred_logit"]),
            r2_score(share["stayshealthy"], share["pred_probit"]),
        ],
        "Brier-score": [
            mean_squared_error(share["stayshealthy"], share["pred_lpm"]),
            mean_squared_error(share["stayshealthy"], share["pred_logit"]),
            mean_squared_error(share["stayshealthy"], share["pred_probit"]),
        ],
        "Pseudo R-squared": [np.nan, logit.prsquared, probit.prsquared],
        "Log-loss": [
            -1 * log_loss(share["stayshealthy"], share["pred_lpm"]),
            -1 * log_loss(share["stayshealthy"], share["pred_logit"]),
            -1 * log_loss(share["stayshealthy"], share["pred_probit"]),
        ],
    },
    index=["LPM", "Logit", "Probit"],
).T.round(3)

### Visual inspection for model comparisons

 a) Comparing predicted probabilities of logit and probit to LPM

First, we need some data manipulation to convert wide to long

In [ ]:
df_plot = pd.melt(
    share[["pred_lpm", "pred_logit", "pred_probit"]].rename(
        columns={"pred_logit": "Logit", "pred_probit": "Probit"}
    ),
    value_vars=["Logit", "Probit"],
    id_vars=["pred_lpm"],
).sort_values(by=["variable"], ascending=False)


In [ ]:
df_plot.head()

In [ ]:
(
    ggplot(data=df_plot)
    + geom_point(aes(x="pred_lpm", y="value", color="variable"), size=0.6, na_rm=True)
    + geom_abline(intercept=0, slope=1, color="green", size=1)
    + labs(
        x="Predicted probability of staying healthy (LPM)", y="Predicted probability"
    )
    + scale_y_continuous(
        expand=[0.00, 0.0], limits=[0, 1], breaks=np.arange(0, 1.1, 0.1)
    )
    + scale_x_continuous(
        expand=[0.00, 0.0], limits=[0, 1], breaks=np.arange(0, 1.1, 0.1)
    )
    + scale_color_manual(name="", values=["blue", "red"])
    + theme_bw()
)

b) Comparing simple LPM and rich LPM's categorization

In [ ]:
lpmbase = smf.ols('stayshealthy ~ smoking', data=share).fit()
share['pred_lpmbase']=lpmbase.predict()

 b1) LPM simple model

In [ ]:
(
    ggplot(
        share,
        aes(x="pred_lpmbase", fill="stayshealthy", y="stat(count/sum(count)))*100"),
    )
    + geom_histogram(
        share[share["stayshealthy"] == 1],
        binwidth=0.05,
        color="blue",
        fill="blue",
        boundary=0.55,
    )
    + geom_histogram(
        share[share["stayshealthy"] == 0],
        binwidth=0.05,
        color="red",
        fill=None,
        boundary=0.55,
    )
    + ylab("Percent")
    + xlab("Fitted values")
    + scale_x_continuous(
        expand=[0.01, 0.01], limits=[0, 1], breaks=np.arange(0, 1.1, 0.2)
    )
    + scale_y_continuous(
        expand=[0.00, 0.00], limits=[0, 80], breaks=np.arange(0, 81, 20)
    )
    + theme_bw()
    + theme(legend_position=(0.3, 0.8))
)

b2) LPM rich model

In [ ]:
(
    ggplot(
        share,
        aes(x="pred_lpm", fill="stayshealthy", y="stat(count/sum(count)))*100"),
    )
    + geom_histogram(
        share[share["stayshealthy"] == 1],
        binwidth=0.05,
        color="blue",
        fill="blue",
        boundary=0.55,
        na_rm=True
    )
    + geom_histogram(
        share[share["stayshealthy"] == 0],
        binwidth=0.05,
        color="red",
        fill=None,
        boundary=0.55,
    )
    + ylab("Percent")
    + xlab("Fitted values")
    + scale_x_continuous(expand=[0.01, 0.01], limits=[0, 1], breaks=np.arange(0, 1.1, 0.2))
    + scale_y_continuous(expand=[0.00, 0.00], limits=[0, 20], breaks=np.arange(0, 21, 4))
    + theme_bw()
)


### Summary statistics on predicted probabilities

#### Task:

Create a CONDITIONAL descriptive statistics on stayhealth for:

    "pred_lpmbase","pred_lpm","pred_logit","pred_probit" 
   
use: "mean","median","min","max","sd" as descriptives

In [ ]:
(
    share.melt(
        id_vars=["stayshealthy"],
        value_vars=["pred_lpmbase", "pred_lpm", "pred_logit", "pred_probit"],
    )
    .groupby(["stayshealthy", "variable"])["value"]
    .agg(["mean", "median", "min", "max", "std"])
)

### Bias and Calibration curve

Lets use the logit model!

 Biased prediction? Calculate bias!
 
 
   Hint: bias = mean(prediction) - mean(actual)
   
   calibration curves -> essentially a scatter-bin!

In [ ]:
breaks = np.array(
    [
        0,
        0.2,
        0.25,
        0.3,
        0.35,
        0.4,
        0.45,
        0.5,
        0.55,
        0.6,
        0.65,
        0.7,
        0.75,
        0.8,
        0.85,
        1.05,
    ]
)

Create binned data:

In [ ]:
share["prob_bin"] = pd.cut(share["pred_logit"], breaks, right=True, include_lowest=True)

In [ ]:
binned_data = (
        share.groupby("prob_bin")
        .agg(
            mean_prob=("pred_logit", "mean"),
            mean_actual=("stayshealthy", "mean"),
            n=("stayshealthy", "size"),
        )
        .reset_index()
    )


In [ ]:
(
    ggplot(binned_data, aes("mean_prob", "mean_actual"))
    + geom_line(color="blue", size=1, show_legend=True)
    + geom_point(
        color="blue",
        size=1,
        alpha=0.7,
        show_legend=False,
        na_rm=True,
    )
    + geom_segment(
        x=min(breaks),
        xend=max(breaks),
        y=min(breaks),
        yend=max(breaks),
        color="red",
        size=0.5,
    )
    + theme_bw()
    + labs(x="Predicted event probability", y="Stays healthy")
    + coord_cartesian(xlim=(0, 1), ylim=(0, 1))
    + expand_limits(x=0.01, y=0.01)
    + scale_y_continuous(expand=(0.01, 0.01), breaks=(np.arange(0, 1.1, 0.1)))
    + scale_x_continuous(expand=(0.01, 0.01), breaks=(np.arange(0, 1.1, 0.1)))
)

#### Task

Do the same calibration curve, but now for LPM rich model\
   Make sure that in the plot you also show the Logit-bias as well

In [ ]:
share["prob_bin"] = pd.cut(share["pred_lpm"], breaks, right=True, include_lowest=True)

In [ ]:
binned_data2 = (
        share.groupby("prob_bin")
        .agg(
            mean_prob_lpm=("pred_lpm", "mean"),
            mean_actual_lpm=("stayshealthy", "mean"),
            n=("stayshealthy", "size"),
        )
        .reset_index()
    )


Join the two data for pretty plot

In [ ]:
binned_data = binned_data.merge(binned_data2, on = "prob_bin")

In [ ]:
(
    ggplot(binned_data)
    + geom_line(aes("mean_prob", "mean_actual"), color="blue", size=1, show_legend=True)
    + geom_point(
        aes("mean_prob", "mean_actual"),
        color="blue",
        size=1,
        alpha=0.7,
        show_legend=False,
        na_rm=True,
    )
    + geom_segment(
        x=min(breaks),
        xend=max(breaks),
        y=min(breaks),
        yend=max(breaks),
        color="red",
        size=0.5,
    )
    + geom_line(
        aes("mean_prob_lpm", "mean_actual_lpm"), color="green", size=1, show_legend=True
    )
    + geom_point(
        aes("mean_prob_lpm", "mean_actual_lpm"),
        color="green",
        size=1,
        alpha=0.7,
        show_legend=False,
        na_rm=True,
    )
    + geom_segment(
        x=min(breaks),
        xend=max(breaks),
        y=min(breaks),
        yend=max(breaks),
        color="red",
        size=0.5,
    )
    + theme_bw()
    + labs(x="Predicted event probability", y="Stays healthy")
    + annotate(
        "text",
        x=(0.2, 0.15),
        y=(0.35, 0.25),
        label=("logit", "LPM"),
        color=("blue", "green"),
    )
    + coord_cartesian(xlim=(0, 1), ylim=(0, 1))
    + expand_limits(x=0.01, y=0.01)
    + scale_y_continuous(expand=(0.01, 0.01), breaks=(np.arange(0, 1.1, 0.1)))
    + scale_x_continuous(expand=(0.01, 0.01), breaks=(np.arange(0, 1.1, 0.1)))
)